# A-gs model and implementation (simulation CO2 and H2O flux)

## Initialize data and model

### Setup and fetch data

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
from Ags_model import runAgs, calc_LE

In [ ]:
import matplotlib.dates as mdates

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')
#NOTE: df_Stor is NOT FILTERED

In [ ]:
from FilterData import Filter_GPP_LE_NEE_VPD 
df_Comb_filter2 = Filter_GPP_LE_NEE_VPD(df_Comb,fqc=None)

In [ ]:
#for manually checking data
#st='2008-01-01'
#ed='2019-12-30'
#df_meteo.loc[st:ed,'L(o)'].plot()
#df_meteo.loc[st:ed,'L(o)corr'].plot()
#df_meteo.loc[st:ed,'Te-L(o)'].plot()
#df_profile.loc[st:ed,'CO2level1'].plot()
#df_profile.loc[st:ed,'Pressure'].plot()
#df_Comb.loc[st:ed,'VPD'].plot()
#df_Comb.loc[st:ed,'Tair'].plot()
#df_Comb.loc[st:ed,'GPP_fqc'].plot()
#df_EC.loc[st:ed,'U-star'].plot()
#df_EC.loc[st:ed,'Mea_Windsp'].plot()

### Import knmi rain

In [ ]:
#datapath = os.path.join('../KNMI_regen')
#print(datapath)
knmi_regen=pd.read_csv('../KNMI_regen/neerslaggeg_KOOTWIJK-RADIO_567.txt',sep=',',header=17)
knmi_regen=knmi_regen.iloc[28000:] #remove dataset before ~november 2007
#knmi_regen=knmi_regen.iloc[31000:] #remove dataset before ~november 2007
knmi_regen['date']=pd.to_datetime(knmi_regen['YYYYMMDD'],yearfirst=True,format='%Y%m%d')
knmi_regen.index=knmi_regen['date']
knmi_regen=knmi_regen.rename(columns={"   RD": "RD", "   SX": "SX"}) #for some reason there's spaces in some of the column names, removing them.
knmi_regen=knmi_regen.astype({"RD": np.intc})
knmi_regen['RD'].plot() #Column RD is is rain daily sum, SX is code related to snow. If unnamed contains 4 spaces it's also a code for snow. STN is station ID.
df_rain=knmi_regen.loc[:,['RD']].resample('3H').ffill()
#df_rain

### Run A-gs model

In [ ]:
# Run A-gs model
#TODO: DEFINITIVELY REPLACE OLD FILTER WITH NP.WHERE FILTER, I TEMP HOTFIXED IT HERE:
an_final,an_umol,rs, ra = runAgs(df_profile_filter,df_Comb_filter2,df_meteo_filter,df_EC_filter,fstr=1.0)

## Calcuate ET

### Assemble dataframe 'df_ET' that will hold output and fill with inputs

In [ ]:
#TODO: UPDATE THIS FUNCTION EVERYWHERE TO ACTUALLY USE FILTERED VALUES!!!!
def init_ETframe(rs_series):
    df_ET = pd.concat([df_meteo_filter['L(o)'],df_meteo_filter['Te-L(o)'],df_profile_filter['Pressure'],df_Comb_filter['VPD'],df_Comb_filter['rH'],df_meteo_filter['P(mast)']],axis=1,sort=False)
    #convert Pressure from hPa to kPa 
    df_ET['p_kPa']=df_ET['Pressure']/10
    df_ET['VPD_adj']=df_ET['VPD'].loc[df_ET['VPD']>0] #some outlier values for VPD are negative, remove from dataset
    df_ET['VPD_adj']=df_ET['VPD_adj']/10  # VPD from df_Comb is in hPa, I need kPa, so hPa/10 = kPa
    df_ET['rs']=rs_series.to_frame(name="rs")
    df_ET['ra']=ra.to_frame(name="ra")
    return df_ET

In [ ]:
df_ET=init_ETframe(rs)

In [ ]:
df_ET=calc_LE(df_ET)

In [ ]:
# Add in precipitation last 3h and 24 hour values so it can be used for filtering
df_ET['last3day_prec']=df_ET['P(mast)'].rolling('72H').sum()

## end of calculate ET

### Manually checking datasets for extreme values

In [ ]:
df_Comb_filter['LE'].plot()

In [ ]:
df_Comb_filter2['LE'].plot()

# fstr analysis

### step 1 assemble dataframe and chop up according to each week

In [ ]:
df_tmp30m=pd.DataFrame()
df_tmp30m['ET']=df_ET['ET_VPD']
df_tmp30m['P(mast)']=df_ET['P(mast)']
df_tmp30m=df_tmp30m.dropna()

In [ ]:
df_tmp30m=pd.DataFrame()
df_tmp30m['ET']=df_ET['ET_VPD']
df_tmp30m['P(mast)']=df_ET['P(mast)']
df_tmp30m=df_tmp30m.dropna()

df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)

#df_tmp30['ET/LE']=df_tmp30['ET']/df_tmp30['LE']
#df_tmp30m['LE/ET']=df_tmp30m['LE']/df_tmp30m['ET'] #<- this is the one we want, observations/model. If obs/model = fstr, then model*fstr= model(obs/model) = obs

df_tmp30m['P_3day']=df_tmp30m['P(mast)'].rolling('72H').sum()
#df_tmp30m=df_tmp30m.loc[df_tmp30m['P_3day']==0]

df_tmp3h=df_tmp30m.resample('3H').mean()#.between_time("09:00", "15:00")
df_tmp3h['P(mast)']=df_tmp30m['P(mast)'].resample('3H').sum() #overwrite the averaged pmast with sum pmast
df_tmp3h['P_3day']=df_tmp3h['P(mast)'].rolling('72H').sum() #overwrite the correct P_3day

#remove rainy days through knmi data
df_tmp3h['RD']=df_rain['RD']
df_tmp3h['RD']=df_tmp3h['RD']/10 #convert from tenths of mm to mm
df_tmp3h=df_tmp3h.loc[df_tmp3h['RD']==0]

#remove rainy days through Pmast
df_rain3h_dailymax=df_tmp3h['P(mast)'].resample('1D').sum().resample('3H').ffill()
df_rain3h_dailymax=df_rain3h_dailymax[:-1] #remove last entry because all 3h frames end on 21:00 not 00:00
df_rain3h_dailymax=df_rain3h_dailymax.rename('P(mast)Dmax')
df_tmp3h['P(mast)Dmax']=df_rain3h_dailymax
df_tmp3h=df_tmp3h.loc[df_tmp3h['P(mast)Dmax']==0]

#manual filter remove bad days
listofdates =  ['2008-08-24',
                '2010-05-11',
                '2010-08-04',
                '2012-06-13',
                '2013-05-10',
                '2013-05-19',
                '2014-06-16',
                '2017-04-22']

#for date in listofdates:
#    df_tmp3h.loc[date+' 00:00':date+' 21:00','ET'] = np.nan
#    df_tmp3h.loc[date+' 00:00':date+' 21:00','LE'] = np.nan

#remove hours outside of 9:00 and 15:00
#df_tmp3h=df_tmp3h.between_time("09:00", "15:00")

#resample to daily
df_tmp1d=df_tmp3h.resample('1D').mean()
df_tmp1d['doy']=df_tmp1d.index.dayofyear
df_tmp1d['P(mast)sum']=df_tmp3h['P(mast)'].resample('1D').sum()
df_tmp1d['LE/ET']=df_tmp1d['LE']/df_tmp1d['ET']

#resample to weekly (7 days)
df_tmp7d=df_tmp3h.resample('7D').mean()
df_tmp7d['LE/ET']=df_tmp7d['LE']/df_tmp7d['ET']

### visualize each year to see whats going wrong

## plotting fstr over time for growthseason

### plotting fstr over time for growthseason some years colored

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=9)]

df_tmp7d['weeknr']=df_tmp7d.index.isocalendar().week

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.0]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,2)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])

colored_years={'2003':'green','2006':'orange','2007':'magenta','2011':'#b62020','2018':'blue','2019':'cyan'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if year in colored_years.keys():
        color=colored_years[year]
        marker='^'
    else:
        color='grey'
        marker=''
    #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_tmp7d.loc[start:end,'weeknr'],df_tmp7d.loc[start:end,'LE/ET'], marker=marker,color=color ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
ax.set_title('fstr (=LEobs/LEsim) on week-basis for 2001-2020 growth seasons (Apr-Sept), 3-hour means for 09:00-15:00, rainy days removed')
#ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

### fstr over time all years colored

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=9)]

df_tmp7d['weeknr']=df_tmp7d.index.isocalendar().week

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.0]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,1.5)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])


for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if int(year)>=2010:
        marker='s'
    else:
        marker='o'

    ax.plot(df_tmp7d.loc[start:end,'weeknr'],df_tmp7d.loc[start:end,'LE/ET'], marker=marker ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
fig.suptitle('fstr (=LEobs/LEsim) on week-basis for april t/m sept, 09:00-15:00, rainy days removed')
ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

### plotting fstr over time some other years colored

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

#remove non-growthseason values
df_tmp7d=df_tmp7d.loc[(df_tmp7d.index.month>=4) & (df_tmp7d.index.month<=9)]

df_tmp7d['weeknr']=df_tmp7d.index.isocalendar().week

#df_tmp7d=df_tmp7d.loc[df_tmp7d['LE/ET']<=1.0]

#df_tmp

#df_tmp['ET/LE'].plot(ylim=(0,15))

fig,ax = plt.subplots()
ax.set_ylim(0,1.5)
ax.set_ylabel('fstr (=LEobs/LEsim)')
ax.set_xlabel('weeknr')

plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

ax2 = ax.twiny()
fig.subplots_adjust(bottom=0.2)

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.15))

# Turn on the frame for the twin axis, but then hide all 
# but the bottom spine
ax2.set_frame_on(True)
ax2.patch.set_visible(False)

ax2.set_xlabel('approx month')
ax2.set_xlim(13,41)
ax2.set_xticks([14, 18, 22, 26,  30, 34, 39])
ax2.set_xticklabels(['Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])

colored_years={'2003':'green','2006':'orange','2011':'#b62020','2013':'purple','2018':'blue'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-01-01 01:00'
    end=year+'-12-23 23:00'
    if year in colored_years.keys():
        color=colored_years[year]
        marker='^'
    else:
        color='grey'
        marker=''
    #marker={'2008':'^','2009':'^','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #colors={'2008':'#b62020','2009':'#cb2424','2010':'^','2011':'^','2012':'o','2013':'^','2014':'o','2015':'^','2016':'o','2017':'o','2018':'^'}
    #df_tmp.loc[start:end,'ET/LE'].plot(x=df_tmp.loc[start:end,'weeknr'],ylim=(0,15))
    ax.plot(df_tmp7d.loc[start:end,'weeknr'],df_tmp7d.loc[start:end,'LE/ET'], marker=marker,color=color ,label=year) #marker=marker[year]
ax.legend(loc=("upper left"))
fig.suptitle('fstr (=LEobs/LEsim) on week-basis for april t/m sept, 09:00-15:00, rainy days removed')
ax.set_title('fstr=0: witing point , fstr=1: no water stress')
ax.grid()

## end of plotting fstr over time for growthseason

## soil moisture anomaly

In [ ]:
#To calculate anomalies monthly precipitation time series from a monthly precipitation time series, 
#subtract the mean value of each month across all years from the original value of that month in each year

#(value soil moisture in that year)-mean soil moisture across all years

#

## soil moisture

## Groundwater

### import groundwater

In [ ]:
#import ground water as sequeltial time series
df_GWS_daily_1 = pd.read_csv('GWS_daily.csv')
df_GWS_daily_1['datetime']=pd.to_datetime(df_GWS_daily_1['datetime'])
df_GWS_daily_1.index=df_GWS_daily_1['datetime']
df_GWS_daily_1.drop(columns=['datetime'],inplace=True) 
df_GWS_daily_1
#columns are datetime (index) depth, depth_anom_all

In [ ]:
#import ground water as sequeltial time series
#import ground water sorted by day of year, each year in column
df_GWS_multi = pd.read_csv('GWS_multi_daily.csv')
df_GWS_multi['yeardate']=pd.to_datetime(df_GWS_multi['yeardate'])
df_GWS_multi.index=df_GWS_multi['yeardate']
df_GWS_multi.drop(columns=['Unnamed: 0'],inplace=True) 
df_GWS_multi
#columns: yeardate (index) monthday (note! a str in form of mm-dd)	depth_2001	depth_2001_diff	depth_2001_anom	depth_2002	depth_2002_diff	depth_2002_anom	depth_2003	depth_2003_diff	depth_2003_anom	...	depth_2016_anom	depth_2017	depth_2017_diff	depth_2017_anom	depth_2018	depth_2018_diff	depth_2018_anom	depth_2019	depth_2019_diff	depth_2019_anom

### end of import groundwater

## Plot GWS over time

In [ ]:
#df_GWS_multi # all years
plt.rcParams.update({'font.size': 16})
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)


fig,ax = plt.subplots()
#ax.set_ylim(0,1.5)
ax.set_ylabel('GW delta since april 1st')
ax.set_xlabel('Date')

#plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
#locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

#fig.subplots_adjust(bottom=0.2)



colored_years={'2003':'green','2006':'orange','2011':'#b62020','2013':'purple','2018':'blue'}

for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2017','2018','2019']:
    start=year+'-04-01'
    end=year+'-10-30'
    if year in colored_years.keys():
        color=colored_years[year]
        marker=''
    else:
        color='grey'
        marker=''
    ax.plot(df_GWS_multi['yeardate'],df_GWS_multi['depth_'+year+'_diff'], marker=marker ,label=year) 
ax.legend(loc=("upper left"))
ax.invert_yaxis()
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=[1,15]))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))

fig.suptitle('Ground water depth relative to depth at start of growth season that year')
ax.set_title('GW_delta = GW_depth(current of that year) - GW_depth(april 1st of that year)')
ax.grid()

In [ ]:
## df_GWS_multi #years that stay down

#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (20,6)

fig,ax = plt.subplots()
#ax.set_ylim(0,1.5)
ax.set_ylabel('GW delta since april 1st')
ax.set_xlabel('Date')

#plt.xticks([x for x in range(14,41)]) #this sets the xticks for the main x-axis with the weeknr
#locs, labels=plt.xticks() #this same function with no argument gets the position and labels and stores them for later

#fig.subplots_adjust(bottom=0.2)

#colored_years={'2003':'green','2008':'orange', 2009'2011':'#b62020','2013':'purple','2018':'blue'}
#2003 2008 2009 2010 (2011) 2013 2014  2018 2019

grey_years=['2001','2002','2004','2005','2006','2007','2010','2011','2012','2015','2016','2017']

#
for year in ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']:
    start=year+'-04-01'
    end=year+'-10-30'
    if year in grey_years:#colored_years.keys():
        marker=''
        ax.plot(df_GWS_multi['yeardate'],df_GWS_multi['depth_'+year+'_diff'], marker=marker, color='grey' ,label='')
    else:
        marker=''
        ax.plot(df_GWS_multi['yeardate'],df_GWS_multi['depth_'+year+'_diff'], marker=marker, zorder=3 ,label=year)
ax.legend(loc=("upper left"))
ax.invert_yaxis()
ax.xaxis.set_minor_locator(mdates.DayLocator())
ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=[1,15]))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d'))

fig.suptitle('Ground water depth relative to depth at start of growth season that year')
ax.set_title('GW_delta = GW_depth(current of that year) - GW_depth(april 1st of that year)')
ax.grid()

## End of Plot GWS over time

## Plotting fstr sensitivity analyses

In [ ]:
plt.rcParams["figure.figsize"] = (6.4, 4.8)

#parallel processing

#df_GWS_multi7d=df_GWS_multi.iloc[:,2:].resample('7D').mean()

df_1d=pd.DataFrame()

df_1d=df_tmp3h.resample('1D').mean()

#df_1d=df_tmp30m.resample('1D').mean()
#df_1d['P(mast)']=df_tmp30m['P(mast)'].resample('1D').sum() #precip needs to be sum, not mean.

df_1d=df_1d.loc[df_1d['P(mast)']==0.0]
df_1d=df_1d.loc[df_1d['RD']==0.0] #knmi rain data

df_1d['LE/ET']=df_1d['LE']/df_1d['ET']
df_1d=df_1d.loc[df_1d['LE/ET']<1.2]

#df_1d=df_1d.merge(df_GWS_multi['GWS_2003_diff'], how='inner',left_index=True, right_index=True)
#df_1d=df_1d.merge(dfGWS['anom'], how='inner',left_index=True, right_index=True)

fig, ax = plt.subplots()

for year in range(2001,2020):
    
    
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    st2="2000-04-01" #due to df_GWS_multi being indexed to the year 2000, we need extra set of st and ed
    ed2="2000-09-30"
    GWS_key='depth_'+str(year)+'_diff' #the column name to ask for
    df_tmp=pd.DataFrame()
    df_tmp[GWS_key]=df_GWS_multi.loc[st2:ed2,[GWS_key]]
    df_tmp['rel_date'] = df_tmp.index.strftime(date_format=str(year)+'-%m-%d')
    df_tmp['rel_date'] = pd.to_datetime(df_tmp['rel_date'],format='%Y-%m-%d')
    df_tmp.index=df_tmp['rel_date']
    #print(df_tmp.index)

    df_tmp=df_tmp.merge(df_1d.loc[st:ed,['LE/ET']], how='inner',left_index=True, right_index=True)
    ax.scatter(df_tmp[GWS_key],df_tmp['LE/ET'],color='b',s=5)
    ax.set_xlabel('GW delta')
    ax.set_ylabel('fstr')
    #ax.set_xlim(-5,90)
    #ax.set_ylim(0,1.2)
    ax.set_title('fstr over groundwater level change since april, \n ')

    #plt.show()

#df_GWS_multi7d

In [ ]:
plt.rcParams["figure.figsize"] = (6.4, 4.8)
plt.rcParams.update({'font.size': 11})
#sequential processing of GWS diff
#df_GWS_multi7d=df_GWS_multi.iloc[:,2:].resample('7D').mean()

df_1d=pd.DataFrame()

df_1d=df_tmp3h.resample('1D').mean()

#df_1d=df_tmp30m.resample('1D').mean()
#df_1d['P(mast)']=df_tmp30m['P(mast)'].resample('1D').sum() #precip needs to be sum, not mean.

df_1d=df_1d.loc[df_1d['P(mast)']==0.0]
df_1d=df_1d.loc[df_1d['RD']==0.0] #knmi rain data

df_1d['LE/ET']=df_1d['LE']/df_1d['ET']
df_1d=df_1d.loc[df_1d['LE/ET']<1.0]


df_1d_seq=pd.DataFrame() #sequential dataframe

for year in range(2001,2020):
    
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    st2="2000-04-01" #due to df_GWS_multi being indexed to the year 2000, we need extra set of st and ed
    ed2="2000-09-30"
    GWS_key='depth_'+str(year)+'_diff' #the column name to ask for
    df_tmp=pd.DataFrame()
    df_tmp[GWS_key]=df_GWS_multi.loc[st2:ed2,[GWS_key]]
    df_tmp['rel_date'] = df_tmp.index.strftime(date_format=str(year)+'-%m-%d')
    df_tmp['rel_date'] = pd.to_datetime(df_tmp['rel_date'],format='%Y-%m-%d')
    df_tmp.index=df_tmp['rel_date']
    df_tmp=df_tmp.merge(df_1d.loc[st:ed,['LE/ET']], how='inner',left_index=True, right_index=True)
    #after this merge is done we can rename the GWS column
    df_tmp.rename(columns={GWS_key:'GWS_diff'},inplace=True)
    #print(df_tmp.columns)
    df_1d_seq = pd.concat([df_1d_seq,df_tmp],axis='index') #vertical join, along index
    df_1d_seq = df_1d_seq.dropna() #remove entries where there are at least one nan

df_1d_seq=df_1d_seq.resample('7D').mean()
df_1d_seq=df_1d_seq.dropna()

fig, ax = plt.subplots()
ax.scatter(df_1d_seq['GWS_diff'],df_1d_seq['LE/ET'],color='b',s=5)

slope, intercept, r_value, p_value, std_err = stats.linregress(df_1d_seq['GWS_diff'],df_1d_seq['LE/ET'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_xlabel('GW change since april 1st of that year [cm]')
ax.set_ylabel(r'$r_{lim}$ [-]')
#ax.set_xlim(-5,90)
#ax.set_ylim(0,1.2)
fig.suptitle(r'$r_{lim}$ over groundwater change since april 1st (weekly average, no rainy days)')
ax.set_title('intercept = {:.3f}, slope = {:.3f}, R2 = {:.3f}'.format(intercept,slope,r_value**2))
ax.legend(loc='upper right')

plt.show()
#df_1d_seq.head(50)


    
#df_GWS_multi7d

In [ ]:
plt.rcParams["figure.figsize"] = (6.4, 4.8)

#sequential processing of days since rain
#df_GWS_multi7d=df_GWS_multi.iloc[:,2:].resample('7D').mean()

df_1d=pd.DataFrame()

df_1d=df_tmp3h.resample('1D').mean()

#df_1d=df_tmp30m.resample('1D').mean()
#df_1d['P(mast)']=df_tmp30m['P(mast)'].resample('1D').sum() #precip needs to be sum, not mean.

###days since rain counting
df_1d['index_shift'] = df_1d.index.shift()
df_1d['timestep']=df_1d['index_shift']-df_1d.index
df_1d['is_1d']=df_1d['timestep']==pd.Timedelta(days=1)
df_1d['days_since_rain']=np.nan #init the new column for assignment

counter=0
for index, row in df_1d.iterrows():
    if row['P(mast)']!=0.0: #if rain isnt zero, reset counting to 0
        counter=0
        df_1d.loc[index,'days_since_rain']=counter
    else: #rain is zero, keep counting up
        if row['is_1d']: #if true, then timedelta is 1 and you keep going, otherwise there's a gap in data and rest to 0
            counter=counter+1
            df_1d.loc[index,'days_since_rain']=counter
            #if index.strftime(date_format='%m-%d')=='04-01': #if it's april first, reset counter
            #    counter=0
            #    df_1d.loc[index,'days_since_rain']=counter
            #else: #otherwise count up
                        
        else: #there is a gap in the data, previous entry not one day before, reset counter to 0
            counter=0
            df_1d.loc[index,'days_since_rain']=counter
            
###days since rain counting end


#filter out rainy days
df_1d=df_1d.loc[df_1d['P(mast)']==0.0]
df_1d=df_1d.loc[df_1d['RD']==0.0] #knmi rain data

df_1d['LE/ET']=df_1d['LE']/df_1d['ET']
df_1d=df_1d.loc[df_1d['LE/ET']<2.0]

df_1d_seq=pd.DataFrame() #sequential dataframe

for year in range(2001,2020):
    
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    st2="2000-04-01" #due to df_GWS_multi being indexed to the year 2000, we need extra set of st and ed
    ed2="2000-09-30"
    GWS_key='depth_'+str(year)+'_diff' #the column name to ask for
    df_tmp=pd.DataFrame()
    df_tmp[GWS_key]=df_GWS_multi.loc[st2:ed2,[GWS_key]]
    df_tmp['rel_date'] = df_tmp.index.strftime(date_format=str(year)+'-%m-%d')
    df_tmp['rel_date'] = pd.to_datetime(df_tmp['rel_date'],format='%Y-%m-%d')
    df_tmp.index=df_tmp['rel_date']
    df_tmp=df_tmp.merge(df_1d.loc[st:ed,['LE/ET','days_since_rain','timestep']], how='inner',left_index=True, right_index=True)
    #print(df_tmp)
    #after this merge is done we can rename the GWS column
    df_tmp.rename(columns={GWS_key:'GWS_diff'},inplace=True)
    #print(df_tmp.columns)
    df_1d_seq = pd.concat([df_1d_seq,df_tmp],axis='index') #vertical join, along index
    df_1d_seq = df_1d_seq.dropna() #remove entries where there are at least one nan

fig, ax = plt.subplots()
ax.scatter(df_1d_seq['days_since_rain'],df_1d_seq['LE/ET'],color='b',s=5)

slope, intercept, r_value, p_value, std_err = stats.linregress(df_1d_seq['days_since_rain'],df_1d_seq['LE/ET'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=-1, c='r',linestyle='dashed',label='1:-1')

ax.set_xlabel('days since rain')
ax.set_ylabel('fstr')
#ax.set_xlim(-5,90)
#ax.set_ylim(0,1.2)
fig.suptitle('fstr over days since rain')
ax.set_title('intercept = {:.3f}, slope = {:.3f}, R2 = {:.3f}'.format(intercept,slope,r_value**2))
ax.legend(loc='upper right')

#plt.show()


#df_1d.head(50)

#df_GWS_multi7d

In [ ]:
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (7, 5)

#sequential processing of days since rain
#df_GWS_multi7d=df_GWS_multi.iloc[:,2:].resample('7D').mean()

df_1d=pd.DataFrame()

df_1d=df_tmp3h.resample('1D').mean()

#df_1d=df_tmp30m.resample('1D').mean()
#df_1d['P(mast)']=df_tmp30m['P(mast)'].resample('1D').sum() #precip needs to be sum, not mean.

###days since rain counting
df_1d['index_shift'] = df_1d.index.shift()
df_1d['timestep']=df_1d['index_shift']-df_1d.index
df_1d['is_1d']=df_1d['timestep']==pd.Timedelta(days=1)
df_1d['days_since_rain']=np.nan #init the new column for assignment

counter=0
for index, row in df_1d.iterrows():
    if row['P(mast)']!=0.0: #if rain isnt zero, reset counting to 0
        counter=0
        df_1d.loc[index,'days_since_rain']=counter
    else: #rain is zero, keep counting up
        if row['is_1d']: #if true, then timedelta is 1 and you keep going, otherwise there's a gap in data and rest to 0
            counter=counter+1
            df_1d.loc[index,'days_since_rain']=counter
            #if index.strftime(date_format='%m-%d')=='04-01': #if it's april first, reset counter
            #    counter=0
            #    df_1d.loc[index,'days_since_rain']=counter
            #else: #otherwise count up
                        
        else: #there is a gap in the data, previous entry not one day before, reset counter to 0
            counter=0
            df_1d.loc[index,'days_since_rain']=counter
            
###days since rain counting end


#filter out rainy days
df_1d=df_1d.loc[df_1d['P(mast)']==0.0]
df_1d=df_1d.loc[df_1d['RD']==0.0] #knmi rain data

df_1d['LE/ET']=df_1d['LE']/df_1d['ET']
df_1d=df_1d.loc[df_1d['LE/ET']<2.0]

df_1d_seq=pd.DataFrame() #sequential dataframe

for year in range(2001,2020):
    
    st=str(year)+"-04-01"
    ed=str(year)+"-09-30"
    st2="2000-04-01" #due to df_GWS_multi being indexed to the year 2000, we need extra set of st and ed
    ed2="2000-09-30"
    GWS_key='depth_'+str(year)+'_diff' #the column name to ask for
    df_tmp=pd.DataFrame()
    df_tmp[GWS_key]=df_GWS_multi.loc[st2:ed2,[GWS_key]]
    df_tmp['rel_date'] = df_tmp.index.strftime(date_format=str(year)+'-%m-%d')
    df_tmp['rel_date'] = pd.to_datetime(df_tmp['rel_date'],format='%Y-%m-%d')
    df_tmp.index=df_tmp['rel_date']
    df_tmp=df_tmp.merge(df_1d.loc[st:ed,['LE/ET','days_since_rain','timestep']], how='inner',left_index=True, right_index=True)
    #print(df_tmp)
    #after this merge is done we can rename the GWS column
    df_tmp.rename(columns={GWS_key:'GWS_diff'},inplace=True)
    #print(df_tmp.columns)
    df_1d_seq = pd.concat([df_1d_seq,df_tmp],axis='index') #vertical join, along index
    df_1d_seq = df_1d_seq.dropna() #remove entries where there are at least one nan

fig, ax = plt.subplots()

df_1d_seq['d_since_int']=df_1d_seq['days_since_rain'].astype(int)

###
#ax.scatter(df_1d_seq['days_since_rain'],df_1d_seq['LE/ET'],color='b',s=5)

sns.boxplot(data=df_1d_seq, x='d_since_int', y='LE/ET', ax=ax, gap=0.5, fliersize=0.01 )

#sns.plot(data=df_1d_seq,ax=ax)

ax.set_xlabel('days since rain')
ax.set_ylabel(r'r$_{lim} [-]$')
#ax.set_xlim(-5,90)
#ax.set_xlim(-1,22.5)
fig.suptitle(r'r$_{lim}$ over days since rain')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
fig.tight_layout()

plt.show()


df_1d_seq.head(5)

#df_GWS_multi7d

In [ ]:
#proof of concept snippet SHIFTING
e=df_tmp30m.loc['2003-01-01':'2003-01-15',['P(mast)']]
e=e.resample('3H').sum()
#e['event']=e.any(e['P(mast)']!=0.0)

e['time_since']=np.nan
counter=0
#e['P(mast)_shift']=e['P(mast)'].shift() #no need for this
e['index_shift']=e.index.shift()
e['timestep']=e['index_shift']-e.index
e['is_30m']=e['timestep']==pd.Timedelta(hours=3)

for index, row in e.iterrows():
    if row['P(mast)']!=0.0: #if rain isnt zero, reset counting to 0
        counter=0
        e.loc[index,'time_since']=counter
    else: #rain is zero, keep counting up
        counter=counter+1
        e.loc[index,'time_since']=counter

e.head(10)

In [ ]:
#proof of concept snippet
e=df_tmp30m.loc['2003-01-01':'2003-01-15',['P(mast)']]
e=e.resample('3H').sum()
#e['event']=e.any(e['P(mast)']!=0.0)

e['time_since']=np.nan
counter=0

for index, row in e.iterrows():
    if row['P(mast)']==0.0:
        counter=counter+1
        e.loc[index,'time_since']=counter
    else:
        counter=0
        e.loc[index,'time_since']=counter

#e.head(40)

In [ ]:
#df_since=
df_tmp3h

In [ ]:
df_tmp=pd.DataFrame()
df_tmp=df_GWS_multi.loc["2000-04-01":"2000-09-30",['depth_'+str(2002)+'_diff']]
df_tmp['str_date'] = df_tmp.index.strftime(date_format=str(2002)+'-%m-%d')
df_tmp.index=df_tmp['str_date']
df_tmp

In [ ]:
#old method
#new location put 88
dir = '../Dinoloket_2024-01-20/Grondwaterstanden_Put/B32F0088001_1_edit.csv'
df_GWS=pd.read_csv(dir,sep=",",index_col=False)
df_GWS=df_GWS.drop(['Locatie','Filternummer','Bijzonderheid','Opmerking'], axis=1)
#df_GWS['Peildatum']=df_GWS['Peildatum']+" 12:00"
df_GWS['datetime']=pd.to_datetime(df_GWS['Peildatum'], dayfirst=True, format='mixed')
df_GWS.index = df_GWS['datetime']
df_GWS=df_GWS.drop(['Peildatum','datetime'], axis=1)
st="2001-01-01"
ed="2019-12-30"
df_GWS=df_GWS.loc[st:ed]
avg_GWS = df_GWS['Stand (cm t.o.v. MV)'].mean()
df_GWS['anom'] = df_GWS.loc[st:ed,'Stand (cm t.o.v. MV)']-avg_GWS

df_GWS_7d=df_GWS.resample('7D').interpolate()
avg_GWS_7d = df_GWS_7d['Stand (cm t.o.v. MV)'].mean()
df_GWS_7d['anom']=df_GWS_7d.loc[st:ed,'Stand (cm t.o.v. MV)']-avg_GWS_7d
df_GWS_7d.index=df_GWS_7d.index.astype('datetime64[s]')

In [ ]:
#old plot
df_tmp30m=pd.DataFrame()
df_tmp30m['ET']=df_ET['ET_VPD']
df_tmp30m=df_tmp30m.dropna()

df_LE=df_Comb['LE'].loc[df_Comb['LE']>=0]
df_tmp30m=df_tmp30m.merge(df_LE, how='inner',left_index=True, right_index=True)
df_1d=df_tmp30m.resample('1D').sum()

df_1d['LE/ET']=df_1d['LE']/df_1d['ET']
df_1d=df_1d.loc[df_1d['LE/ET']<1.0]

fig,ax = plt.subplots()

st="2011-04-01"
ed="2011-09-30"


df_1d=df_1d.merge(df_GWS['anom'], how='inner',left_index=True, right_index=True)
#df_1d=df_1d.merge(dfGWS['anom'], how='inner',left_index=True, right_index=True)

ax.scatter(df_1d['anom'],df_1d['LE/ET'])

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['anom'],left_index=True, right_index=True,direction='backward')
#df_1d=df_1d.merge(dfGWS['anom'], how='inner',left_index=True, right_index=True)

ax.scatter(df_tmp7d_1['anom'],df_tmp7d_1['LE/ET'])

ax.set_ylim(0,1)
ax.set_title('weekly average fstr (only 9:00-15:00, no rainy days) over groundwater anomaly')
#fstr over groundwater change since april 1st (weekly average)'
ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('groundwater anomaly (cm) (positive value means deeper)')

import scipy.stats as stats

df_tmp7d_1=df_tmp7d_1.dropna()

slope, intercept, r_value, p_value, std_err = stats.linregress(df_tmp7d_1['anom'], df_tmp7d_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')
#df_1d=df_1d.merge(dfGWS['anom'], how='inner',left_index=True, right_index=True)

ax.scatter(df_tmp7d_1['Stand (cm t.o.v. MV)'],df_tmp7d_1['LE/ET'])

ax.set_ylim(0,1)
ax.set_title('7-day average fstr (only 9:00-15:00, no rainy days) over groundwater depth')
ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('groundwater depth (cm) (positive value means deeper)')

import scipy.stats as stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df_tmp7d_1['Stand (cm t.o.v. MV)'], df_tmp7d_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
#df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')


df_tmp7d_1=df_tmp7d.merge(df_soil_7d['SM-003'], how='inner',left_index=True, right_index=True)

ax.scatter(df_tmp7d_1['SM-003'],df_tmp7d_1['LE/ET'])

ax.set_ylim(0,1.2)
ax.set_title('7-day average fstr (only 9:00-15:00, no rainy days) over Soil moisture (at 3cm)')
ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('soil moisture at 3cm depth (% volume)')

import scipy.stats as stats

df_tmp_7d_1=df_tmp_7d_1.dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(df_tmp7d_1['SM-003'], df_tmp7d_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
#df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')


df_tmp7d_1=df_tmp7d.merge(df_soil_7d['SM-Lit'], how='inner',left_index=True, right_index=True)

ax.scatter(df_tmp7d_1['SM-Lit'],df_tmp7d_1['LE/ET'])

ax.set_ylim(0,1.2)
ax.set_title('7-day average fstr (only 9:00-15:00, no rainy days) over Soil moisture (Litter)')
ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('soil moisture at Litter (% volume)')

import scipy.stats as stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df_tmp7d_1['SM-Lit'], df_tmp7d_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
df_soil_7d_sm3_mean=df_soil_7d['SM-003'].mean()
df_soil_7d_lit_mean=df_soil_7d['SM-Lit'].mean()
df_soil_7d['SM-003_anom']=df_soil_7d['SM-003']-df_soil_7d_sm3_mean
df_soil_7d['SM-Lit_anom']=df_soil_7d['SM-Lit']-df_soil_7d_lit_mean

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
#df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')


df_tmp7d_1=df_tmp7d.merge(df_soil_7d['SM-003_anom'], how='inner',left_index=True, right_index=True)

ax.scatter(df_tmp7d_1['SM-003_anom'],df_tmp7d_1['LE/ET'])

ax.set_ylim(0,1.2)
ax.set_title('7-day average fstr (only 9:00-15:00, no rainy days) over Soil moisture Anomaly (3cm depth)')
ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('soil moisture anomaly at 3cm depth (% volume)')

import scipy.stats as stats

df_tmp7d_1=df_tmp7d_1.dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(df_tmp7d_1['SM-003_anom'], df_tmp7d_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
### HEREEEE
SM= pd.read_csv('Soil_moisture.csv')
SM['datetime']=pd.to_datetime(SM['datetime'])
SM.index=SM['datetime']
SM.drop(columns='datetime',inplace=True)
#SM has columns SM-003	SM-020	SM-050	SM-100	'SM_l1' (i.e. 0-50cm)	'SM_l2' (i.e. 50-100)
#and runs from 2005-04-11 00:00:00 2020-12-31 23:30:00 and freq 30 min
#SM=SM.resample('7D').mean()
SM

In [ ]:
df_tmp7d

In [ ]:
df_tmp1dc_day

In [ ]:
#SM_1=SM.reset_index()
#df_tmp7d_1=df_tmp7d.reset_index()
#SM_1 = pd.concat([SM,df_tmp7d_1],axis=1)
SM_1 = SM.resample('D').mean()

SM_1 = pd.concat([SM_1,df_tmp1dc_day],axis=1)
#SM_1=SM_1.loc[:,['SM_l1','SM_l2','fstr',P]]
SM_1

In [ ]:
df_tmp1dc_day

In [ ]:
#fstr max over 1 day, fstr values can be bigger than 1, day-values (08-18:00) and all
df_tmp1d      =pd.read_csv('./fstrOutput/NoRain/fstr_1d.csv')
df_tmp1d.index=pd.to_datetime(df_tmp1d['Unnamed: 0'])
df_tmp1d.index.name='date'
df_tmp1d.drop(columns='Unnamed: 0',inplace=True)
df_tmp1d_day  =pd.read_csv('./fstrOutput/NoRain/fstrMax_1d_day.csv')
df_tmp1d_day.index=pd.to_datetime(df_tmp1d_day['Unnamed: 0'])
df_tmp1d_day.index.name='date'
df_tmp1d_day.drop(columns='Unnamed: 0',inplace=True)

#fstr max over 1 day, values of fstr are clamped between 0 and 1, day-values (08-18:00) and all
df_tmp1dc     =pd.read_csv('./fstrOutput/NoRain/fstr_1d_clamped.csv')
df_tmp1dc.index=pd.to_datetime(df_tmp1dc['Unnamed: 0'])
df_tmp1dc.index.name='date'
df_tmp1dc.drop(columns='Unnamed: 0',inplace=True)
df_tmp1dc_day =pd.read_csv('./fstrOutput/NoRain/fstr_1d_clamped_day.csv')
df_tmp1dc_day.index=pd.to_datetime(df_tmp1dc_day['Unnamed: 0'])
df_tmp1dc_day.index.name='date'
df_tmp1dc_day.drop(columns='Unnamed: 0',inplace=True)

#fstr max over 1 day, values of fstr are clamped between 0 and 1, day-values (08:00-18:00) only
df_tmp1dcm    =pd.read_csv('./fstrOutput/NoRain/fstrMax_1d_clamped_day.csv')
df_tmp1dcm.index=pd.to_datetime(df_tmp1dcm['Unnamed: 0'])
df_tmp1dcm.index.name='date'
df_tmp1dcm.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
SM_1=pd.concat([SM_1,df_tmp1dc_day],axis=1)
SM_1

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)
plt.rcParams.update({'font.size': 14})

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
#df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')


#df_tmp7d_1=df_tmp7d.merge(SM_1['SM_l1'], how='inner',left_index=True, right_index=True)

ax.scatter(SM_1['SM_l1'],SM_1['fstr'],marker='.')

#ax.set_ylim(0,1.2)
ax.set_xlim(0.02,0.12)

ax.set_ylabel(r'$r_{lim}$ daily mean')
ax.set_xlabel('daily mean soil moisture (mean of 0-100cm layer) [%])')

import scipy.stats as stats

SM_1=SM_1.dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(SM_1['SM_l1'], SM_1['fstr']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
ax.set_title('daily average r_lim (only 9:00-15:00, no rainy days) over Soil moisture (0-100cm) \n intercept = 0.656 , slope =  0.885 , R2 = 0.0004 ')
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
#plt.rcParams["figure.figsize"] = plt.rcParamsDefault
#plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["figure.figsize"] = (6,6)

#df_tmp7d_1=df_tmp7d.copy()
#df_tmp1d_1=df_tmp1d_1.loc[df_tmp1d_1['LE/ET']<1.0]

fig,ax = plt.subplots()

#st="2011-04-01"
#ed="2011-09-30"

#df_GWS_7d['Stand (cm t.o.v. MV)']
#df_tmp7d_1=pd.merge_asof(df_tmp7d,df_GWS_7d['Stand (cm t.o.v. MV)'],left_index=True, right_index=True,direction='backward')


#df_tmp7d_1=df_tmp7d.merge(SM_1['SM_l1'], how='inner',left_index=True, right_index=True)

ax.scatter(SM_1['SM_l2'],SM_1['LE/ET'])

ax.set_ylim(0,1.2)
ax.set_xlim(0.03,0.12)

ax.set_ylabel('fstr 7day mean')
ax.set_xlabel('soil moisture layer2 50-100cm (1/100 % volume)')

import scipy.stats as stats

SM_1=SM_1.dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(SM_1['SM_l2'], SM_1['LE/ET']) #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
ax.set_title('7-day average fstr (only 9:00-15:00, no rainy days) over Soil moisture (50-100cm) \n intercept = 0.546 , slope =  2.95 , R2 = 0.00267 ')
print('R2: ',r_value**2)
print('slope, intercept:', slope, intercept)
ax.axline((0.0,intercept),slope=slope,c='r',label='slope')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

In [ ]:
df_soil_7d=df_soil.loc['2001-06-21':'2019-07-25'].resample('7D').mean()#['SM-Lit']
df_soil_7d['SM-003'].plot()

In [ ]:
df_tmp7d

In [ ]:
df_tmp7d.index

In [ ]:
df_soil_7d.columns